In [64]:
import pandas as pd
import json

In [65]:
input_file = "../data/WikiEvents/train_coref.jsonl"
with open(input_file) as f:
    lines = f.read().splitlines()
    df_inter = pd.DataFrame(lines)
    df_inter.columns = ['json_element']
    df_inter['json_element'].apply(json.loads)
    df_co = pd.json_normalize(df_inter['json_element'].apply(json.loads))
input_file = "../data/WikiEvents/train.jsonl"
with open(input_file) as f:
    lines = f.read().splitlines()
    df_inter = pd.DataFrame(lines)
    df_inter.columns = ['json_element']
    df_inter['json_element'].apply(json.loads)
    df_we = pd.json_normalize(df_inter['json_element'].apply(json.loads))    
with open("../data/WikiEvents/coref_mapping.json") as f:
    coref_mapping = json.load(f)

**Ziel ist es für trigger und arguments die passenden indizes zu haben um sie mit den Ergebnissen des Modells zu vergleichen. Daher muss für jedes argument noch der span eingefügt werden. Für Coref Matching müssen noch die coreferences eingefügt werden.** 

In [66]:
df_we.iloc[205].entity_mentions

[{'id': 'scenario_en_kairos_35-T1',
  'sent_idx': 0,
  'start': 0,
  'end': 1,
  'entity_type': 'GPE',
  'mention_type': 'UNK',
  'text': 'German'},
 {'id': 'scenario_en_kairos_35-T2',
  'sent_idx': 0,
  'start': 1,
  'end': 2,
  'entity_type': 'ORG',
  'mention_type': 'UNK',
  'text': 'police'},
 {'id': 'scenario_en_kairos_35-T3',
  'sent_idx': 0,
  'start': 6,
  'end': 7,
  'entity_type': 'PER',
  'mention_type': 'UNK',
  'text': 'suspect'},
 {'id': 'scenario_en_kairos_35-T4',
  'sent_idx': 0,
  'start': 14,
  'end': 15,
  'entity_type': 'WEA',
  'mention_type': 'UNK',
  'text': 'explosives'},
 {'id': 'scenario_en_kairos_35-T5',
  'sent_idx': 0,
  'start': 21,
  'end': 22,
  'entity_type': 'VEH',
  'mention_type': 'UNK',
  'text': 'train'},
 {'id': 'scenario_en_kairos_35-T6',
  'sent_idx': 0,
  'start': 22,
  'end': 23,
  'entity_type': 'FAC',
  'mention_type': 'UNK',
  'text': 'station'},
 {'id': 'scenario_en_kairos_35-T7',
  'sent_idx': 0,
  'start': 24,
  'end': 25,
  'entity_type

In [67]:
df_we.iloc[205].event_mentions

[{'id': 'scenario_en_kairos_35-E2',
  'event_type': 'Conflict.Attack.Unspecified',
  'trigger': {'start': 58, 'end': 59, 'text': 'attack', 'sent_idx': 1},
  'arguments': [{'entity_id': 'scenario_en_kairos_35-T15',
    'role': 'Place',
    'text': 'Madrid'},
   {'entity_id': 'scenario_en_kairos_35-T16',
    'role': 'Instrument',
    'text': 'bombs'}]},
 {'id': 'scenario_en_kairos_35-E3',
  'event_type': 'Life.Die.Unspecified',
  'trigger': {'start': 70, 'end': 71, 'text': 'claimed', 'sent_idx': 1},
  'arguments': [{'entity_id': 'scenario_en_kairos_35-T18',
    'role': 'Victim',
    'text': 'lives'}]},
 {'id': 'scenario_en_kairos_35-E4',
  'event_type': 'Justice.ArrestJailDetain.Unspecified',
  'trigger': {'start': 86, 'end': 87, 'text': 'detained', 'sent_idx': 3},
  'arguments': [{'entity_id': 'scenario_en_kairos_35-T21',
    'role': 'Detainee',
    'text': 'men'}]},
 {'id': 'scenario_en_kairos_35-E5',
  'event_type': 'Justice.ArrestJailDetain.Unspecified',
  'trigger': {'start': 121, '

In [68]:
from collections import defaultdict
import json
df_we = pd.read_json('../data/WikiEvents/train.json').set_index('doc_id')
df = pd.read_json("../data/WikiEvents/train_coref.json").set_index('doc_key')

coref_mapping = defaultdict(dict)
for doc_id, row in df.iterrows():
    for cluster in row.clusters:
        for item in cluster:
            coref_mapping[doc_id][item] = cluster
for doc_id, row in df_we.iterrows():
    for event in row.event_mentions:
        for arg in event['arguments']:
            if arg['entity_id'] not in coref_mapping[doc_id].keys():
                coref_mapping[doc_id][arg['entity_id']] = arg['entity_id']

with open("../data/WikiEvents/coref_mapping.json", "w") as f:
    json.dump(coref_mapping,f)

In [69]:
with open("../data/WikiEvents/coref_mapping.json") as f:
    coref_mapping = json.load(f)

In [70]:
for idx,row in df_we[:2].iterrows():
    
    for em in row.event_mentions:
        for arg in em['arguments']:
            coref_ids = coref_mapping[idx][arg['entity_id']]
            corefs = []
            for c in coref_ids:
                for ent in row.entity_mentions:
                    if arg['entity_id'] == ent['id']:
                        arg['span'] = (ent['start'],ent['end'])
                        continue
                    if c == ent['id']:
                        coref = {
                            'entity_id':ent['id'],
                            'start':ent['start'],
                            'end':ent['end'],
                            'text': ent['text']
                        }
                        corefs.append(coref)
            arg['corefs'] = corefs

In [71]:
df_we.iloc[1].event_mentions

[{'id': 'scenario_en_kairos_65-E1',
  'event_type': 'Conflict.Attack.Unspecified',
  'trigger': {'start': 50, 'end': 51, 'text': 'attack', 'sent_idx': 1},
  'arguments': []},
 {'id': 'scenario_en_kairos_65-E2',
  'event_type': 'Life.Injure.Unspecified',
  'trigger': {'start': 62, 'end': 63, 'text': 'injured', 'sent_idx': 2},
  'arguments': [{'entity_id': 'scenario_en_kairos_65-T13',
    'role': 'Victim',
    'text': 'Terry Duffield',
    'span': (59, 61),
    'corefs': [{'entity_id': 'scenario_en_kairos_65-T37',
      'start': 195,
      'end': 196,
      'text': 'Duffield'},
     {'entity_id': 'scenario_en_kairos_65-T80',
      'start': 388,
      'end': 389,
      'text': 'his'},
     {'entity_id': 'scenario_en_kairos_65-T101',
      'start': 532,
      'end': 533,
      'text': 'He'},
     {'entity_id': 'scenario_en_kairos_65-T110',
      'start': 620,
      'end': 621,
      'text': 'Duffield'},
     {'entity_id': 'scenario_en_kairos_65-T79',
      'start': 383,
      'end': 384,
 